<a href="https://colab.research.google.com/github/AlbertoVentura8/LimpiezaDatos/blob/main/MarchMad_Limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**En este apartado, me voy a centrar en la limpieza del dataset.**


1.   Voy a dar acceso a un drive donde está almacenado y donde almacenaremos el
resultado del dataset.
2.   Eliminaremos una serie de columnas que hacen que el conjunto de datos tenga una dimensión poco manejable, y lo vamos a reducir para que sea tratable y poder sacar algunas conclusiones.

In [1]:
# Doy permisos para importar los archivos desde un drive y poder guardar y cargar los archivos ahí
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Hacemos la carga del dataset. Dataset previamente subido a GoogleColab.
import pandas as pd
NCAA_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TFM/March_Madness_df.csv")
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (6966, 52)


,Unnamed: 0.1,Unnamed: 0,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),...,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Net Rating
0,0,4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,...,30.03,14.67,8.82,8.61,23.45,26.61,18.51,17.84,13.59,34.2
1,1,6,2021,Baylor,Big 12 Conference,67.4,125.0,91.1,17.0,17.6,...,39.18,15.00,7.14,5.21,30.09,27.69,14.23,16.34,11.65,33.9
2,2,7,2015,Wisconsin,Big Ten Conference (Big 10),58.7,129.0,95.2,21.8,18.3,...,28.34,29.46,9.31,11.50,33.03,20.78,22.11,11.68,12.40,33.8
3,3,8,2018,Villanova,Big East Conference,68.7,127.8,94.0,17.9,16.6,...,27.17,19.54,9.99,4.45,29.62,19.16,19.99,17.28,13.94,33.8
4,4,9,2011,Ohio State,Big Ten Conference (Big 10),63.8,125.0,91.5,17.1,20.1,...,18.74,15.64,14.62,9.35,35.97,17.52,15.71,16.02,14.77,33.5


In [3]:
#En esta línea eliminamos todas la columnas que tengan la palabra Raw, para reducir el dataset inicial.
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Raw')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()


Tamaño del dataset: (6966, 52)


,Unnamed: 0.1,Unnamed: 0,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),...,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Net Rating
0,0,4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,...,30.03,14.67,8.82,8.61,23.45,26.61,18.51,17.84,13.59,34.2
1,1,6,2021,Baylor,Big 12 Conference,67.4,125.0,91.1,17.0,17.6,...,39.18,15.00,7.14,5.21,30.09,27.69,14.23,16.34,11.65,33.9
2,2,7,2015,Wisconsin,Big Ten Conference (Big 10),58.7,129.0,95.2,21.8,18.3,...,28.34,29.46,9.31,11.50,33.03,20.78,22.11,11.68,12.40,33.8
3,3,8,2018,Villanova,Big East Conference,68.7,127.8,94.0,17.9,16.6,...,27.17,19.54,9.99,4.45,29.62,19.16,19.99,17.28,13.94,33.8
4,4,9,2011,Ohio State,Big Ten Conference (Big 10),63.8,125.0,91.5,17.1,20.1,...,18.74,15.64,14.62,9.35,35.97,17.52,15.71,16.02,14.77,33.5


In [4]:
#Hacemos lo mismo que en la línea anterior para eliminar los rankings.
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Rank')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (6966, 52)


,Unnamed: 0.1,Unnamed: 0,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),...,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Net Rating
0,0,4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,...,30.03,14.67,8.82,8.61,23.45,26.61,18.51,17.84,13.59,34.2
1,1,6,2021,Baylor,Big 12 Conference,67.4,125.0,91.1,17.0,17.6,...,39.18,15.00,7.14,5.21,30.09,27.69,14.23,16.34,11.65,33.9
2,2,7,2015,Wisconsin,Big Ten Conference (Big 10),58.7,129.0,95.2,21.8,18.3,...,28.34,29.46,9.31,11.50,33.03,20.78,22.11,11.68,12.40,33.8
3,3,8,2018,Villanova,Big East Conference,68.7,127.8,94.0,17.9,16.6,...,27.17,19.54,9.99,4.45,29.62,19.16,19.99,17.28,13.94,33.8
4,4,9,2011,Ohio State,Big Ten Conference (Big 10),63.8,125.0,91.5,17.1,20.1,...,18.74,15.64,14.62,9.35,35.97,17.52,15.71,16.02,14.77,33.5


In [5]:
# Con estas dos limpiezas, ya hemos pasado de un dataset de 144 columnas, a uno de 75
# Vamos a ver los tipos de nuestras columnas
print(NCAA_df.dtypes)
tipo_resumen = NCAA_df.dtypes.value_counts()
print("Tipos de Datos más comunes: ", tipo_resumen)

Unnamed: 0.1                         int64
Unnamed: 0                           int64
Season                               int64
Team Name                           object
Conference                          object
Adjusted Temo                      float64
Adjusted Offensive Efficiency      float64
Adjusted Defensive Efficiency      float64
Avg Possession Length (Offense)    float64
Avg Possession Length (Defense)    float64
eFGPct                             float64
TOPct                              float64
ORPct                              float64
OffFT                              float64
DefFT                              float64
Tempo                              float64
AdjTempo                           float64
OE                                 float64
AdjOE                              float64
DE                                 float64
AdjDE                              float64
AdjEM                              float64
FG2Pct                             float64
FG3Pct     

In [6]:
# Vamos a intentar reducir más la cantidad de inputs
NCAA_df = NCAA_df.iloc[:, :-4]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (6966, 48)


,Unnamed: 0.1,Unnamed: 0,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),...,Experience,Bench,Centerpts,CenterOR,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR
0,0,4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,...,1.60,22.86,17.93,37.86,30.03,14.67,8.82,8.61,23.45,26.61
1,1,6,2021,Baylor,Big 12 Conference,67.4,125.0,91.1,17.0,17.6,...,2.01,28.57,17.98,33.47,39.18,15.00,7.14,5.21,30.09,27.69
2,2,7,2015,Wisconsin,Big Ten Conference (Big 10),58.7,129.0,95.2,21.8,18.3,...,2.08,21.24,28.32,21.39,28.34,29.46,9.31,11.50,33.03,20.78
3,3,8,2018,Villanova,Big East Conference,68.7,127.8,94.0,17.9,16.6,...,1.33,25.88,16.09,38.82,27.17,19.54,9.99,4.45,29.62,19.16
4,4,9,2011,Ohio State,Big Ten Conference (Big 10),63.8,125.0,91.5,17.1,20.1,...,1.60,18.20,25.21,41.65,18.74,15.64,14.62,9.35,35.97,17.52


In [7]:
# Seguimos eliminando columnas, ahora vamos con la 3 columna, que es similar a la segunda, Historical Conference
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Historical')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (6966, 48)


,Unnamed: 0.1,Unnamed: 0,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),...,Experience,Bench,Centerpts,CenterOR,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR
0,0,4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,...,1.60,22.86,17.93,37.86,30.03,14.67,8.82,8.61,23.45,26.61
1,1,6,2021,Baylor,Big 12 Conference,67.4,125.0,91.1,17.0,17.6,...,2.01,28.57,17.98,33.47,39.18,15.00,7.14,5.21,30.09,27.69
2,2,7,2015,Wisconsin,Big Ten Conference (Big 10),58.7,129.0,95.2,21.8,18.3,...,2.08,21.24,28.32,21.39,28.34,29.46,9.31,11.50,33.03,20.78
3,3,8,2018,Villanova,Big East Conference,68.7,127.8,94.0,17.9,16.6,...,1.33,25.88,16.09,38.82,27.17,19.54,9.99,4.45,29.62,19.16
4,4,9,2011,Ohio State,Big Ten Conference (Big 10),63.8,125.0,91.5,17.1,20.1,...,1.60,18.20,25.21,41.65,18.74,15.64,14.62,9.35,35.97,17.52


In [8]:
# Seguimos eliminando columnas, ahora vamos con unas columnas que contienen NST que no nos aportan demasiada información
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('NST')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (6966, 48)


,Unnamed: 0.1,Unnamed: 0,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),...,Experience,Bench,Centerpts,CenterOR,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR
0,0,4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,...,1.60,22.86,17.93,37.86,30.03,14.67,8.82,8.61,23.45,26.61
1,1,6,2021,Baylor,Big 12 Conference,67.4,125.0,91.1,17.0,17.6,...,2.01,28.57,17.98,33.47,39.18,15.00,7.14,5.21,30.09,27.69
2,2,7,2015,Wisconsin,Big Ten Conference (Big 10),58.7,129.0,95.2,21.8,18.3,...,2.08,21.24,28.32,21.39,28.34,29.46,9.31,11.50,33.03,20.78
3,3,8,2018,Villanova,Big East Conference,68.7,127.8,94.0,17.9,16.6,...,1.33,25.88,16.09,38.82,27.17,19.54,9.99,4.45,29.62,19.16
4,4,9,2011,Ohio State,Big Ten Conference (Big 10),63.8,125.0,91.5,17.1,20.1,...,1.60,18.20,25.21,41.65,18.74,15.64,14.62,9.35,35.97,17.52


In [9]:
# Seguimos eliminando columnas, ahora vamos con puntos ofensivos y defensivos, ya que nos interesa ir a los porcentajes sobretodo
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('PtFG')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (6966, 48)


,Unnamed: 0.1,Unnamed: 0,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),...,Experience,Bench,Centerpts,CenterOR,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR
0,0,4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,...,1.60,22.86,17.93,37.86,30.03,14.67,8.82,8.61,23.45,26.61
1,1,6,2021,Baylor,Big 12 Conference,67.4,125.0,91.1,17.0,17.6,...,2.01,28.57,17.98,33.47,39.18,15.00,7.14,5.21,30.09,27.69
2,2,7,2015,Wisconsin,Big Ten Conference (Big 10),58.7,129.0,95.2,21.8,18.3,...,2.08,21.24,28.32,21.39,28.34,29.46,9.31,11.50,33.03,20.78
3,3,8,2018,Villanova,Big East Conference,68.7,127.8,94.0,17.9,16.6,...,1.33,25.88,16.09,38.82,27.17,19.54,9.99,4.45,29.62,19.16
4,4,9,2011,Ohio State,Big Ten Conference (Big 10),63.8,125.0,91.5,17.1,20.1,...,1.60,18.20,25.21,41.65,18.74,15.64,14.62,9.35,35.97,17.52


In [10]:
# Seguimos eliminando columnas, ahora vamos con las columnas que contienen Rate
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Rate')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (6966, 48)


,Unnamed: 0.1,Unnamed: 0,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),...,Experience,Bench,Centerpts,CenterOR,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR
0,0,4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,...,1.60,22.86,17.93,37.86,30.03,14.67,8.82,8.61,23.45,26.61
1,1,6,2021,Baylor,Big 12 Conference,67.4,125.0,91.1,17.0,17.6,...,2.01,28.57,17.98,33.47,39.18,15.00,7.14,5.21,30.09,27.69
2,2,7,2015,Wisconsin,Big Ten Conference (Big 10),58.7,129.0,95.2,21.8,18.3,...,2.08,21.24,28.32,21.39,28.34,29.46,9.31,11.50,33.03,20.78
3,3,8,2018,Villanova,Big East Conference,68.7,127.8,94.0,17.9,16.6,...,1.33,25.88,16.09,38.82,27.17,19.54,9.99,4.45,29.62,19.16
4,4,9,2011,Ohio State,Big Ten Conference (Big 10),63.8,125.0,91.5,17.1,20.1,...,1.60,18.20,25.21,41.65,18.74,15.64,14.62,9.35,35.97,17.52


In [11]:
# Seguimos eliminando columnas, ahora vamos con unas columnas que contienen información sobre los entrenadores
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Coach|coach')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (6966, 48)


,Unnamed: 0.1,Unnamed: 0,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),...,Experience,Bench,Centerpts,CenterOR,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR
0,0,4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,...,1.60,22.86,17.93,37.86,30.03,14.67,8.82,8.61,23.45,26.61
1,1,6,2021,Baylor,Big 12 Conference,67.4,125.0,91.1,17.0,17.6,...,2.01,28.57,17.98,33.47,39.18,15.00,7.14,5.21,30.09,27.69
2,2,7,2015,Wisconsin,Big Ten Conference (Big 10),58.7,129.0,95.2,21.8,18.3,...,2.08,21.24,28.32,21.39,28.34,29.46,9.31,11.50,33.03,20.78
3,3,8,2018,Villanova,Big East Conference,68.7,127.8,94.0,17.9,16.6,...,1.33,25.88,16.09,38.82,27.17,19.54,9.99,4.45,29.62,19.16
4,4,9,2011,Ohio State,Big Ten Conference (Big 10),63.8,125.0,91.5,17.1,20.1,...,1.60,18.20,25.21,41.65,18.74,15.64,14.62,9.35,35.97,17.52


In [12]:
# Seguimos eliminando columnas, ahora vamos con las columnas que contienen Pts
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Pts|pts')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (6966, 47)


,Unnamed: 0.1,Unnamed: 0,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),...,EffectiveHeight,Experience,Bench,CenterOR,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR
0,0,4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,...,1.18,1.60,22.86,37.86,30.03,14.67,8.82,8.61,23.45,26.61
1,1,6,2021,Baylor,Big 12 Conference,67.4,125.0,91.1,17.0,17.6,...,-0.56,2.01,28.57,33.47,39.18,15.00,7.14,5.21,30.09,27.69
2,2,7,2015,Wisconsin,Big Ten Conference (Big 10),58.7,129.0,95.2,21.8,18.3,...,5.05,2.08,21.24,21.39,28.34,29.46,9.31,11.50,33.03,20.78
3,3,8,2018,Villanova,Big East Conference,68.7,127.8,94.0,17.9,16.6,...,-0.43,1.33,25.88,38.82,27.17,19.54,9.99,4.45,29.62,19.16
4,4,9,2011,Ohio State,Big Ten Conference (Big 10),63.8,125.0,91.5,17.1,20.1,...,0.03,1.60,18.20,41.65,18.74,15.64,14.62,9.35,35.97,17.52


In [13]:
# Vamos a ver que variedad de años tenemos, para quedarnos con los últimos años
NCAA_df['Season'].value_counts()

,count
Season,
2023,363
2024,361
2022,358
2021,356
2019,353
2020,353
2017,351
2014,351
2016,351


In [14]:
#Como vemos todos los años están bastante parecidos, y tenemos datos desde el 2002. Vamos a eliminar los datos del 2002, 2003 y 2004 para reducir algo mas el dataset y usar los últimos 20 años
NCAA_df = NCAA_df[~NCAA_df['Season'].isin([2002, 2003, 2004])]
NCAA_df['Season'].value_counts()

,count
Season,
2023,363
2024,361
2022,358
2021,356
2019,353
2020,353
2017,351
2014,351
2016,351


In [15]:
# Vamos a ver el número de líneas que nos quedan
print("Tamaño del dataset:", NCAA_df.shape)

Tamaño del dataset: (6966, 47)


In [16]:
# Vamos a eliminar todas las lineas que tengan algún tipo de caracter en blanco
NCAA_df = NCAA_df.replace(pd.NA, regex=True).dropna()
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()


Tamaño del dataset: (6966, 47)


,Unnamed: 0.1,Unnamed: 0,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),...,EffectiveHeight,Experience,Bench,CenterOR,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR
0,0,4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,...,1.18,1.60,22.86,37.86,30.03,14.67,8.82,8.61,23.45,26.61
1,1,6,2021,Baylor,Big 12 Conference,67.4,125.0,91.1,17.0,17.6,...,-0.56,2.01,28.57,33.47,39.18,15.00,7.14,5.21,30.09,27.69
2,2,7,2015,Wisconsin,Big Ten Conference (Big 10),58.7,129.0,95.2,21.8,18.3,...,5.05,2.08,21.24,21.39,28.34,29.46,9.31,11.50,33.03,20.78
3,3,8,2018,Villanova,Big East Conference,68.7,127.8,94.0,17.9,16.6,...,-0.43,1.33,25.88,38.82,27.17,19.54,9.99,4.45,29.62,19.16
4,4,9,2011,Ohio State,Big Ten Conference (Big 10),63.8,125.0,91.5,17.1,20.1,...,0.03,1.60,18.20,41.65,18.74,15.64,14.62,9.35,35.97,17.52


In [17]:
# Por último, vamos a renombrar el df, para tenerlo listo para los siguientes pasos
March_Madness_df = NCAA_df
March_Madness_df.to_csv("/content/drive/MyDrive/Colab Notebooks/TFM/March_Madness_df.csv")